In [14]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import sklearn.datasets as skd
import sklearn.metrics as skm
from time import *
from tqdm import tqdm
from torch_geometric.utils import to_dense_adj
import os
import torch
from sgl import LearnGraphTopolgy
os.getcwd()
dataset = os.path.join(os.getcwd(),'Citeseer')
dataset
from torch_geometric.datasets import Planetoid
dataset= Planetoid(root=dataset, name='Citeseer')
x = dataset[0].x.detach().cpu().numpy()
labels = dataset[0].y
NO_OF_CLASSES =  len(set(np.array(dataset[0].y)))






adj = to_dense_adj(dataset[0].edge_index)
adj = adj[0]
labels = dataset[0].y
labels = labels.numpy()

X = dataset[0].x
X = X.to_dense()
N = X.shape[0]
NO_OF_CLASSES =  len(set(np.array(dataset[0].y)))

print(X.shape, adj.shape)

nn = int(1*N)
X = X[:nn,:]
adj = adj[:nn,:nn]
A = adj[:nn,:nn]
AT= torch.transpose(A,0,1)
labels = labels[:nn]
print(dataset[0])


def get_laplacian(adj):
    b=torch.ones(adj.shape[0])
    return torch.diag(adj@b)-adj

theta = get_laplacian(adj)
print(theta.shape)

torch.Size([3327, 3703]) torch.Size([3327, 3327])
Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])
torch.Size([3327, 3327])


In [2]:
X.shape

torch.Size([3327, 3703])

In [4]:
S = np.dot(X, X.T)
Sinv = np.linalg.pinv(S)
Sinv.shape[1]

3327

In [ ]:
S = np.dot(X, X.T)

# estimate underlying graph
sgl = LearnGraphTopolgy(S, maxiter=250, record_objective = True, record_weights = True)
# graph = sgl.learn_k_component_graph(w0 = 'qp', k=5, beta=1 )
graph = sgl.learn_k_component_graph(k=6, beta=2)

# build network
A = graph['adjacency']
G = nx.from_numpy_matrix(A)
print('Graph statistics:')
print('Nodes: ', G.number_of_nodes(), 'Edges: ', G.number_of_edges() )

 20%|███████████████▍                                                               | 49/250 [37:55<2:22:53, 42.66s/it]

In [20]:

A = graph['adjacency']
G = nx.from_numpy_matrix(A)
print('Graph statistics:')
print('Nodes: ', G.number_of_nodes(), 'Edges: ', G.number_of_edges() )

Graph statistics:
Nodes:  3327 Edges:  32761


In [21]:
def nb_connected_component(L):
    return np.sum(np.linalg.eigh(L)[0]<10**-12)
L = graph['laplacian']
nb_connected_component(L)

41

In [22]:
from scipy import sparse
from random import sample
adj2=graph['adjacency']
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(X.shape[1], 64)
        self.conv2 = GCNConv(64, NO_OF_CLASSES)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)
    
    
####### NO output layer is written
NO_OF_NODES = X.shape[0]
Wc=adj2
Wc=sparse.csr_matrix(Wc)
Wc = Wc.tocoo()
row = torch.from_numpy(Wc.row).to(torch.long)
col = torch.from_numpy(Wc.col).to(torch.long)
edge_index_coarsen2 = torch.stack([row, col], dim=0)
edge_weight = torch.from_numpy(Wc.data)

device = torch.device('cpu')
Y = labels
labels_coarse=torch.Tensor(Y).type(torch.long)
Wc=Wc.toarray()
model=Net().to(device)
device = torch.device('cpu')
lr=0.1
decay=0.01
features = torch.Tensor(X)
try:
  X=np.array(features.todense())
except:
  X = np.array(features)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=decay)
p = X.shape[0]
x=range(0,int(p*0.9))
xx=range(int(p*0.9),p)
from datetime import datetime
def train():
    model.train()
    optimizer.zero_grad()
    out = model(torch.Tensor(X).to(device),edge_index_coarsen2)
    loss = F.nll_loss(out[x], labels_coarse[x])
    loss.backward()
    optimizer.step()
    return loss
now1 = datetime.now()
losses=[]
for epoch in range(100):
    loss=train()
    losses.append(loss)
    if(epoch%10==0):
        print(f'Epoch: {epoch:03d},loss: {loss:.4f}')
now2 = datetime.now()
pred=model(torch.Tensor(X).to(device),edge_index_coarsen2).argmax(dim=1)

def train_accuracy():
    model.eval()
    correct = (pred[xx] == labels_coarse[xx]).sum()
    acc = int(correct) /len(xx)
    return acc

zz=sample(range(0, int(NO_OF_NODES)), NO_OF_NODES)
Wc=sparse.csr_matrix(adj2)
Wc = Wc.tocoo()
row = torch.from_numpy(Wc.row).to(torch.long)
col = torch.from_numpy(Wc.col).to(torch.long)
edge_index_coarsen = torch.stack([row, col], dim=0)
edge_weight = torch.from_numpy(Wc.data)
pred=model(torch.Tensor(X),edge_index_coarsen).argmax(dim=1)
pred=np.array(pred)
correct =(pred[xx]==labels[xx]).sum()
acc=int(correct)/len(xx)
acc

Epoch: 000,loss: 1.7927
Epoch: 010,loss: 1.7393
Epoch: 020,loss: 1.7285
Epoch: 030,loss: 1.7468
Epoch: 040,loss: 1.6697
Epoch: 050,loss: 1.6380
Epoch: 060,loss: 1.6499
Epoch: 070,loss: 1.6410
Epoch: 080,loss: 1.6373
Epoch: 090,loss: 1.6214


0.3153153153153153

In [20]:
Y.shape

(3327,)

In [23]:
labels

array([3, 1, 5, ..., 3, 1, 5], dtype=int64)